In [5]:
import pandas as pd
import numpy as np
import talib as ta

# Carregar dados CSV
df = pd.read_csv('BTCUSDT_bybit_data_1h.csv')

# Convertendo o timestamp para o formato datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Parâmetros de Entrada
ema_short_length = 11
ema_long_length = 55
rsi_length = 22
volatility_period = 14
atr_multiplier = 2.5
macd_short = 15
macd_long = 34
macd_signal = 11
adx_length = 14
adx_threshold = 12
bb_length = 14
bb_multiplier = 1.7
lateral_threshold = 0.005
spread = 0.5  # Spread em porcentagem

# Filtrar o período de datas
start_date = '2023-01-01'
end_date = '2023-12-31'
df = df[(df['timestamp'] >= start_date) & (df['timestamp'] <= end_date)]

# Indicadores
df['ema_short'] = ta.EMA(df['close'], timeperiod=ema_short_length)
df['ema_long'] = ta.EMA(df['close'], timeperiod=ema_long_length)
df['rsi'] = ta.RSI(df['close'], timeperiod=rsi_length)
df['atr'] = ta.ATR(df['high'], df['low'], df['close'], timeperiod=volatility_period)

# Cálculo do MACD
df['macd'], df['macd_signal'], df['macd_hist'] = ta.MACD(df['close'], fastperiod=macd_short, slowperiod=macd_long, signalperiod=macd_signal)

# Cálculo do ADX
df['adx'] = ta.ADX(df['high'], df['low'], df['close'], timeperiod=adx_length)

# Bollinger Bands
df['bb_upper'], df['bb_middle'], df['bb_lower'] = ta.BBANDS(df['close'], timeperiod=bb_length, nbdevup=bb_multiplier, nbdevdn=bb_multiplier)

# Detecção de lateralização
df['bandwidth'] = (df['bb_upper'] - df['bb_lower']) / df['bb_middle']
df['is_lateral'] = df['bandwidth'] < lateral_threshold

# Aplicar Spread no preço
df['ask_price'] = df['close'] * (1 + spread / 100.0)
df['bid_price'] = df['close'] * (1 - spread / 100.0)

# Condições de tendência
df['trending_market'] = df['adx'] > adx_threshold

# Condições de entrada long
df['long_condition'] = (df['ema_short'] > df['ema_long']) & (df['rsi'] < 60) & (df['macd_hist'] > 0.5) & df['trending_market'] & ~df['is_lateral']

# Condições de entrada short
df['short_condition'] = (df['ema_short'] < df['ema_long']) & (df['rsi'] > 40) & (df['macd_hist'] < -0.5) & df['trending_market'] & ~df['is_lateral']

# Condições de reversão de mercado lateral (mean reversion)
df['long_lateral'] = (df['close'] < df['bb_lower']) & (df['close'].shift(1) < df['bb_lower']) & df['is_lateral']
df['short_lateral'] = (df['close'] > df['bb_upper']) & (df['close'].shift(1) > df['bb_upper']) & df['is_lateral']

# Filtrar apenas as linhas com sinais de transação (entrada)
transactions = df[(df['long_condition'] | df['short_condition'] | df['long_lateral'] | df['short_lateral'])]

# Exibir as transações
print(transactions[['timestamp', 'close', 'long_condition', 'short_condition', 'long_lateral', 'short_lateral']])


                  timestamp    close  long_condition  short_condition  \
4819601 2023-01-01 18:00:00  16595.5           False            False   
4819801 2023-01-01 19:00:00  16608.0           False            False   
4820001 2023-01-01 20:00:00  16608.0           False            False   
4820201 2023-01-01 21:00:00  16611.0           False            False   
4820202 2023-01-01 20:00:00  16608.0            True            False   
...                     ...      ...             ...              ...   
6601391 2023-12-31 00:00:00  42399.0            True            False   
6601398 2023-12-30 17:00:00  42525.6            True            False   
6601399 2023-12-30 16:00:00  42436.4            True            False   
6601592 2023-12-31 00:00:00  42399.0            True            False   
6602399 2023-12-30 21:00:00  42338.1           False             True   

         long_lateral  short_lateral  
4819601         False           True  
4819801         False           True  
482000